# Some notes on processing WRF data
*Prepared by Alex Rybchuk*

The Weather Research and Forecasting (WRF) model is used for regional analysis of the atmosphere (spatial scale is often approximately the size of a U.S. state). It is used in all sorts of applications, ranging from weather forecasting to wildfire modeling to renewable energy analysis.

In [1]:
### Import libraries
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

Output data is usually saved as files that start with `wrfout` or `auxhist`, and it is usually saved in the netCDF format.

In [2]:
### Load data
## List file names
wrf_files = ['wrfout_d02_2019-09-10_00:00:00',
             'wrfout_d02_2019-09-10_06:00:00']

## Read multiple files into one Xarray Dataset 
ds = xr.open_mfdataset(wrf_files,
                        parallel=True,     # Do a "lazy" read in case the files are collectively too large for memory
                        engine='netcdf4',  # Specify that these are netCDF4 files
                        combine='nested',  
                        concat_dim='Time') # Combine files along the Time dimension

By default, `wrfout` files contain many different variables.

In [3]:
ds

<xarray.Dataset>
Dimensions:                (Time: 2, bottom_top: 53, bottom_top_stag: 54, pft_fraction_stag: 17, seed_dim_stag: 2, soil_layers_stag: 4, south_north: 258, south_north_stag: 259, west_east: 465, west_east_stag: 466)
Coordinates:
    XLAT                   (Time, south_north, west_east) float32 dask.array<chunksize=(1, 258, 465), meta=np.ndarray>
    XLONG                  (Time, south_north, west_east) float32 dask.array<chunksize=(1, 258, 465), meta=np.ndarray>
    XTIME                  (Time) datetime64[ns] dask.array<chunksize=(1,), meta=np.ndarray>
    XLAT_U                 (Time, south_north, west_east_stag) float32 dask.array<chunksize=(1, 258, 466), meta=np.ndarray>
    XLONG_U                (Time, south_north, west_east_stag) float32 dask.array<chunksize=(1, 258, 466), meta=np.ndarray>
    XLAT_V                 (Time, south_north_stag, west_east) float32 dask.array<chunksize=(1, 259, 465), meta=np.ndarray>
    XLONG_V                (Time, south_north_stag, west_east) float32 dask.array<chunksize=(1, 259, 465), meta=np.ndarray>
Dimensions without coordinates: Time, bottom_top, bottom_top_stag, pft_fraction_stag, seed_dim_stag, soil_layers_stag, south_north, south_north_stag, west_east, west_east_stag
Data variables: (12/104)
    Times                  (Time) |S19 dask.array<chunksize=(1,), meta=np.ndarray>
    ZNU                    (Time, bottom_top) float32 dask.array<chunksize=(1, 53), meta=np.ndarray>
    ZNW                    (Time, bottom_top_stag) float32 dask.array<chunksize=(1, 54), meta=np.ndarray>
    ZS                     (Time, soil_layers_stag) float32 dask.array<chunksize=(1, 4), meta=np.ndarray>
    DZS                    (Time, soil_layers_stag) float32 dask.array<chunksize=(1, 4), meta=np.ndarray>
    VAR_SSO                (Time, south_north, west_east) float32 dask.array<chunksize=(1, 258, 465), meta=np.ndarray>
    ...                     ...
    C4H                    (Time, bottom_top) float32 dask.array<chunksize=(1, 53), meta=np.ndarray>
    C3F                    (Time, bottom_top_stag) float32 dask.array<chunksize=(1, 54), meta=np.ndarray>
    C4F                    (Time, bottom_top_stag) float32 dask.array<chunksize=(1, 54), meta=np.ndarray>
    PCB                    (Time, south_north, west_east) float32 dask.array<chunksize=(1, 258, 465), meta=np.ndarray>
    PC                     (Time, south_north, west_east) float32 dask.array<chunksize=(1, 258, 465), meta=np.ndarray>
    SST                    (Time, south_north, west_east) float32 dask.array<chunksize=(1, 258, 465), meta=np.ndarray>
Attributes: (12/131)
    TITLE:                            OUTPUT FROM WRF V4.3 MODEL
    START_DATE:                      2019-09-09_06:00:00
    SIMULATION_START_DATE:           2019-08-30_00:00:00
    WEST-EAST_GRID_DIMENSION:        466
    SOUTH-NORTH_GRID_DIMENSION:      259
    BOTTOM-TOP_GRID_DIMENSION:       54
    ...                              ...
    ISLAKE:                          21
    ISICE:                           15
    ISURBAN:                         13
    ISOILWATER:                      14
    HYBRID_OPT:                      0
    ETAC:                            0.0

Variables are calculated on grids that are either unstaggered or staggered. For example, temperature `T` is calculated only using unstaggered coordinates. It's dimensions are `Time`, `bottom_top`, `south_north`, and `west_east`.

In [4]:
ds['T'].dims

('Time', 'bottom_top', 'south_north', 'west_east')

Other times, variables will have one staggered dimension. For example, zonal (x-) winds `U` are staggered along the x-axis. They have the `west_east_stag` dimension. 

In [5]:
ds['U'].dims

('Time', 'bottom_top', 'south_north', 'west_east_stag')

Staggered dimensions are one element longer than unstaggered dimensions. 

In [6]:
len(ds['bottom_top']), len(ds['bottom_top_stag'])

(53, 54)

Staggering can be annoying to deal with, especially when trying to calculate quantities like wind speed. To calculate wind speed, destagger `U` along the x-axis and `V` along the y-axis. To destagger, all we need to do is average the fields on the staggered grid. We'll also help Xarray out by explicitly renaming the destaggered dimension.

In [7]:
## Calculate how many grid cells are along the staggered dimension
nx_stag = len(ds['west_east_stag'])
ny_stag = len(ds['south_north_stag'])

## Horizontally destagger winds
ds['U_DESTAG'] = 0.5 * (
                    ds['U'].isel(west_east_stag=slice(0,nx_stag-1)).rename({'west_east_stag':'west_east'}) \
                    + ds['U'].isel(west_east_stag=slice(1,nx_stag)).rename({'west_east_stag':'west_east'}))

ds['V_DESTAG'] = 0.5 * (
                    ds['V'].isel(south_north_stag=slice(0,ny_stag-1)).rename({'south_north_stag':'south_north'}) \
                    + ds['V'].isel(south_north_stag=slice(1,ny_stag)).rename({'south_north_stag':'south_north'}))

## Calculate wind speed
ds['WSPD'] = np.sqrt(ds['U_DESTAG']**2 + ds['V_DESTAG']**2)

The same approach can be used to destagger fields in the vertical dimension.

Speaking of the vertical dimension, WRF uses pressure fields ("eta levels") as its vertical coordinate system instead of height in meters. To calculate the height of the vertical grid in meters, add the perturbation geopotential `PH` to the base-state geopotential `PHB` and divide by the gravitational acceleration.

Note: Technically, the height of the vertical coordinate slightly changes (~1%) every timestep as pressure fields move around. In practice, we just calculate the height at one moment in time and treat those heights as constant.

In [8]:
### Calculate height relative to sea level
grav = 9.81    # m/s2
height_stag = (ds['PH'].isel(Time=-1) + ds['PHB'].isel(Time=-1)) / grav
height_stag.isel(bottom_top_stag=0).values

array([[  5.0244884,   4.791814 ,   3.7197638, ...,   0.       ,
          0.       ,   0.       ],
       [  5.266056 ,   5.266056 ,   4.266686 , ...,   0.       ,
          0.       ,   0.       ],
       [  5.565803 ,   5.7262635,   4.8451734, ...,   0.       ,
          0.       ,   0.       ],
       ...,
       [368.86786  , 380.32898  , 386.69995  , ...,   0.       ,
          0.       ,   0.       ],
       [354.03885  , 363.60907  , 370.58325  , ...,   0.       ,
          0.       ,   0.       ],
       [349.09692  , 350.86594  , 356.1308   , ...,   0.       ,
          0.       ,   0.       ]], dtype=float32)

The above calculation was for height relative to sea level. To calculate the height relative to local terrain, subtrain terrain height `HGT`.

In [9]:
height_stag_rel = height_stag - ds['HGT'].isel(Time=-1)
height_stag_rel.isel(bottom_top_stag=0).values

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

Just like earlier, we can destagger this variable, but now in the vertical dimension.

In [10]:
### Calculate unstaggered heights
nz_stag = len(ds['bottom_top_stag'])
height_unstag = 0.5 * (
                    height_stag.isel(bottom_top_stag=slice(0,nz_stag-1)).rename({'bottom_top_stag':'bottom_top'}) \
                    + height_stag.isel(bottom_top_stag=slice(1,nz_stag)).rename({'bottom_top_stag':'bottom_top'}))
height_unstag_rel = height_unstag - ds['HGT'].isel(Time=-1)

In [11]:
height_unstag_rel.isel(bottom_top=0).values

array([[5.248349 , 5.2575903, 5.266016 , ..., 5.2844286, 5.28485  ,
        5.2856297],
       [5.247963 , 5.2593174, 5.2686696, ..., 5.285765 , 5.2866426,
        5.287128 ],
       [5.2475715, 5.2587085, 5.2665744, ..., 5.286949 , 5.287714 ,
        5.288153 ],
       ...,
       [4.923126 , 4.924591 , 4.927002 , ..., 5.1277146, 5.1282067,
        5.128978 ],
       [4.9198914, 4.919861 , 4.920532 , ..., 5.1277637, 5.128212 ,
        5.128599 ],
       [4.92041  , 4.9201965, 4.920227 , ..., 5.127088 , 5.127605 ,
        5.1280265]], dtype=float32)

These are some basic things that impact all users of WRF data. In your specific research endeavors, you may run into other pecuiliarities with the data.

All of the code used in the examples here have been written "from scratch". NCAR has put together the `wrf-python` library ([here](https://wrf-python.readthedocs.io/en/latest/)) that calculate a number of diagnostic variables, such as CAPE and cloud top temperature. You may find this library useful in your research, or you may find it overly cumbersome. In all likelihood, your Conda environment doesn't have `wrf-python`, so if you would like this tool, talk to your graduate student mentor to get a hand installing it.